# Generating answers w/ Source Citation

In [2]:
import os
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from IPython.display import Markdown, display, update_display

In [3]:
load_dotenv()
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
openai = OpenAI()
embeddings = OpenAIEmbeddings()

In [10]:
if os.path.exists(f"{db_name}"):
    vectorstore = Chroma(embedding_function=embeddings, persist_directory=f"{db_name}")
    print(f"Vectorstore loaded with {vectorstore._collection.count()} documents")
else:
    print("Deleting any reference to the existing db..")
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
    print(
        "Vector store doesn't exist. Please run the Day 5 notebook first to create Chroma Vector DB & injest the data."
    )

Vectorstore loaded with 123 documents


In [11]:
system_message = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Use the following markdown format to answer the question along with the Source used to generate the answer, add inline citation for each sentence & add end of the answer citations:
'CEO of Insurellm is Avery Lancaster [[1]](Source Link 1). Who is also a co-founder [[2]](Source Link 2)
Citations: (Note: No duplicates allowed in the below list)

[1 - Source Title 1](Link 1)
[2 - Source Title 2](Link 2)
...
[n - Source Title n](Link n)'

Example answer:
'CEO of Insurellm is Avery Lancaster [[1]](knowledge-base\\company\\about.md). Who is also a co-founder [[2]](knowledge-base\\employees\\Avery Lancaster.md)
Citations:

[1 - About Company](knowledge-base\\company\\about.md)
[2 - Avery Lancaster employees](knowledge-base\\employees\\Avery Lancaster.md)'

Important Note: Have unique end of the answer citations. Don't give duplicate citation numbers for the same source link, reuse the same citation number if the same source link is referenced multiple times.
'"""